In [ ]:
from teach.dataset.definitions import Definitions
definitions = Definitions(version="2.0")
print("Agent IDs to agents: ", definitions.map_agents_id2info)
print("Status IDs to names: ", definitions.map_status_id2name)

In [ ]:
import json
def print_utterances_and_dialog_acts(game_dict, definitions, da_dict):
    # interactions = game_dict["tasks"][0]["episodes"][0]["interactions"]
    # print(interactions)

    for task in game_dict["tasks"]:
        for episode in task["episodes"]:
            for interaction in episode["interactions"]:
                if interaction.get("da_metadata"):
                    # output_str = ""
                    # output_str += definitions.map_agents_id2info[interaction["agent_id"]]["agent_name"].ljust(15, " ")
                    # output_str += "Utterance: " + interaction["utterance"] + "\n"
                    # # output_str += "".ljust(15, " ") + "Corrected: " + interaction["corrected_utterance"] + "\n"
                    # output_str += "".ljust(15, " ") + "DAs with segments: \n"
                    agent_name = definitions.map_agents_id2info[interaction["agent_id"]]["agent_name"]
                    for idx in range(len(interaction["da_metadata"]["das"])):
                        # interaction["da_metadata"]["text_segments"] and interaction["da_metadata"]["das"] are lists of length 3
                        # If an utterance has fewer than 3 DAs then the extra segments and DAs are empty
                        # No utterance has more than 3 DAs
                        utt_segment = interaction["da_metadata"]["text_segments"][idx]
                        da = interaction["da_metadata"]["das"][idx].lower()

                        if da_dict.get(da) is None:
                            da_dict[da] = {}
                        da_dict[da]["utterances"] = da_dict[da].get("utterances", []) + [utt_segment]
                        da_dict[da]["actor"] = da_dict[da].get("actor", []) + [agent_name]
                    #     if len(da) > 0:
                    #         output_str += "".ljust(30, " ") + da + ": " + utt_segment + "\n"
                    # print(output_str + "\n")
    return da_dict

In [ ]:
da_dict = {}

paths = ["games/train", "games/valid_seen", "games/valid_unseen"]
data_dir = "/tmp/teach-dataset"
for path in paths:
    path = os.path.join(data_dir, path)
    # train_path = os.path.join(data_dir, "games/valid_seen")
    # train_path = os.path.join(data_dir, "games/valid_unseen")
    print(path)
    for file in os.listdir(path=path):
        if file.endswith('.json'):
            print(file)
            f = os.path.join(path, file)
            with open(f) as h:
                game_dict = json.load(h)
                # print(game_dict)
                if da_dict.get(path.split("/")[-1]) is None:
                    da_dict[path.split("/")[-1]] = {}
                da_dict[path.split("/")[-1]] = print_utterances_and_dialog_acts(game_dict, definitions, da_dict[path.split("/")[-1]])



/tmp/teach-dataset/games/train
e93c7148850f2f47_2243.game.json
0e137379e033a29e_5b37.game.json
081da8ed289d8db8_60bf.game.json
307f7a5c7564459e_e868.game.json
4e751ee7c37c15bf_8885.game.json
65bf3effcd66614c_6bb8.game.json
4c6348317575edee_1419.game.json
d54ecb14fdda356e_6d46.game.json
22709334946cf784_3d3f.game.json
c1c7fbddb4c7b2f5_0a77.game.json
f8c6564165e92a55_d1b9.game.json
ae7cdd042e3974b9_5551.game.json
e86c3799e62dfbd7_bf14.game.json
e8b5a025c8195338_1d27.game.json
6a17757da98f27b3_67d1.game.json
fe6b7cf10fc0ed82_4b67.game.json
0fc552ac1a3cfcb8_b491.game.json
f3757d30a1bd4765_63d9.game.json
3714918fd5efe467_6bd4.game.json
39d5a56196f52b83_fed1.game.json
ace4dabd9ac967cd_9424.game.json
438f4a51e6d45115_9cb6.game.json
ac55cf858902e28f_260c.game.json
b63c06c50b3c5e73_4808.game.json
7bb34ac28d66f590_9051.game.json
8a82a98d91fec42e_8ee9.game.json
ab2a07eab2b458d7_783e.game.json
07d4c1f1cef30af4_0c5b.game.json
c3afba31d738f0cd_28a2.game.json
4be09183db6ae879_8861.game.json
41c94fc36

In [ ]:
# da_dict["Greetings/Salutations"]
# set(da_dict['Greetings/Salutations'])
da_dict.keys()


dict_keys(['train', 'valid_seen', 'valid_unseen'])

In [ ]:
import pandas as pd

df = None
for split_type, da in da_dict.items():
    for dialog_act, da_map in da.items():
        utterances = da_map["utterances"]
        actors = da_map["actor"]
        if dialog_act == '':
            continue
        # utts = list(set(utterances))
        if df is not None:
            new_da_df = pd.DataFrame({
                "dialog_act": dialog_act, 
                "utterances": utterances, 
                "split": split_type,
                "actor": actors
                })
            df = pd.concat([df, new_da_df])
        else:
            df = pd.DataFrame({
                "dialog_act": dialog_act, 
                "utterances": utterances, 
                "split": split_type,
                "actor": actors
                })

df = df.drop_duplicates()
df.to_csv("teach-da.csv", index=False)

In [ ]:
grouped_df = df.groupby(["dialog_act", "actor"]).agg({"utterances": lambda x: "\n".join(x[:10])}).reset_index()
grouped_df.to_csv("teach-da-grouped.csv", index=False)

/var/folders/wx/k6_t0dmd7msdv162l8lv24y00000gn/T/ipykernel_26723/2668569784.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  grouped_df = df.groupby(["dialog_act", "actor"]).agg({"utterances": lambda x: "\n".join(x[:10])}).reset_index()


In [51]:
import pandas as pd
df = pd.read_csv("teach-da.csv")

grouped_df = pd.read_csv("TeachDa Dialog Acts - teach-da-grouped (1).csv")
grouped_df = grouped_df.drop(columns=["utterances"])
df = df.join(grouped_df.set_index(["dialog_act", "actor"]), on=["dialog_act", "actor"])

# df = df.rename(columns={"new name": "intent"})
df["intent"] = df.apply(lambda x: x["intent"] if x["include?"] == "y" else "other", axis=1)

df["actor"] = df["actor"].apply(lambda x: "user" if x == "Commander" else "agent")
df.to_csv("teach_da_final.csv", index=False)

In [60]:
grouped_df = pd.read_csv("TeachDa Dialog Acts - teach-da-grouped (1).csv")
# df["next steps"].value_counts()
grouped_df

,dialog_act,actor,utterances,include?,intent,next steps
0,acknowledge,User,we are done for now\nFinished.\nAll tasks comp...,y,Affirm,NaN
1,acknowledge,Agent,thank you\nok\nThe lettuce has been sliced.\nT...,y,Affirm,cleanup
2,affirm,User,"yes\nOoh maybe\nyes!\nyes,\nYes\nyes plate\nYe...",y,Affirm,NaN
3,affirm,Agent,yes\nok\nYep.\nye\nyea we need anothyer\nYeah\...,y,Affirm,NaN
4,alternatequestions,User,butterknife or sharp\nare you done\nIs it too ...,n,NaN,consider whether to merge in other intents
5,alternatequestions,Agent,"mug is clean, should i make coffee?\nOr the ot...",n,NaN,NaN
6,confirm,User,do you see it?\nyou cleaned all the pots?\na s...,y,Confirm,cleanup
7,confirm,Agent,could it be in a cabinet?\nDoes not pour coffe...,y,Confirm,cleanup
8,deny,User,"no\nno, that was all,\nNo\nnot a couch.\nI don...",y,Deny,NaN
9,deny,Agent,no knife\nNope\nno\nnot yet\nNo dish\nNo.\nNo\...,y,Deny,cleanup


In [59]:

df = pd.read_csv("teach-da.csv")
df

,dialog_act,utterances,split,actor
0,requestforinstruction,directions please,train,Driver
1,requestforinstruction,how can I help?,train,Driver
2,requestforinstruction,What should I do today?,train,Driver
3,requestforinstruction,Next?,train,Driver
4,requestforinstruction,hey how can I help?,train,Driver
...,...,...,...,...
20449,feedbacknegative,its not yet complete.,valid_unseen,Commander
20450,feedbacknegative,its not complete yet,valid_unseen,Commander
20451,feedbacknegative,the task is not complete yet,valid_unseen,Commander
20452,feedbacknegative,it is stillnot complete,valid_unseen,Commander


In [ ]:
grouped_df = pd.read_csv("TeachDa Dialog Acts - teach-da-grouped.csv")

In [13]:
# Switchboard corpus
import pandas as pd
df = pd.read_csv("Google_ Dialog Acts - Sheet1.csv")


In [14]:
df = df[~df["intent"].isna()]

In [23]:
df

,Unnamed: 0,name,act_tag,intent,next steps,example,train_count,full_count,Amol,Chandra,Stephen,Nachiketa,Nikhil,Comments
1,2,Acknowledge (Backchannel),b,Affirm,NaN,Uh-huh.,37096,38298,NaN,Y,"Y \n\nMerging with ""yes"": Could depend on cont...",Y\n\n- tag with affirm/yes intent,Yes,[Nikhil] I don't think this is same as yes. We...
3,4,Agree/Accept,aa,Affirm,NaN,That's exactly it.,10820,11133,NaN,Y,Y,Y\n\n- tag with affirm/yes intent,Yes,NaN
8,9,Yes answers,ny,Affirm,NaN,Yes.,2934,3034,NaN,Y,Y,Y\n\n- tag with affirm/yes intent,Yes,NaN
9,10,Conventional-closing,fc,Goodbye,NaN,"Well, it's been nice talking to you.",2486,2582,NaN,Y,Y,Y\n\n- tag with goodbye,Yes,NaN
12,13,No answers,nn,Deny,NaN,No.,1340,1377,NaN,Y,Y,Y \n\n- tag with no/deny intent,Yes,NaN
13,14,Response Acknowledgement,bk,Affirm,NaN,"Oh, okay.",1277,1306,NaN,Y,Y,Y\n\n- tag with affirm/yes intent,Yes,"[Nikhil] Should this be part of ""continue"" but..."
17,18,Backchannel in question form,bh,Confirm,NaN,Is that right?,1019,1053,NaN,Y,Y,Excluded:\n - Relevant in speech not in text,Yes,NaN
20,21,Affirmative non-yes answers,na,Affirm,NaN,It is.,836,847,NaN,Y,Y,Y\n\n- tag with affirm/yes intent,Maybe,NaN
27,28,Reject,ar,Deny,NaN,"Well, no",338,346,NaN,Y,Y,Y\n\n- tag with no/deny intent,Yes,NaN
28,29,Negative non-no answers,ng,Feedback Negative,Cleanup,"Uh, not a whole lot.",292,302,NaN,NaN,Y,Excluded:\n - Relevant in speech not in text,NaN,NaN


In [24]:
_DAMSL_ACT_TAGS = [
    "ad",
    "qo",
    "qy",
    "arp_nd",
    "sd",
    "h",
    "bh",
    "no",
    "^2",
    "^g",
    "ar",
    "aa",
    "sv",
    "bk",
    "fp",
    "qw",
    "b",
    "ba",
    "t1",
    "oo_co_cc",
    "+",
    "ny",
    "qw^d",
    "x",
    "qh",
    "fc",
    'fo_o_fw_"_by_bc',
    "aap_am",
    "%",
    "bf",
    "t3",
    "nn",
    "bd",
    "ng",
    "^q",
    "br",
    "qy^d",
    "fa",
    "^h",
    "b^m",
    "ft",
    "qrr",
    "na",
]

In [27]:
swda_da_tags = df["act_tag"].tolist()
swda_da_tag_indexes = [_DAMSL_ACT_TAGS.index(da) for da in swda_da_tags]

In [4]:
from datasets import load_dataset

dataset = load_dataset("swda")

Dataset swda downloaded and prepared to /Users/nikhil/.cache/huggingface/datasets/swda/default/0.0.0/9af7f63229aca2a0d84408dd35ceb640b18d13f36d4b6e668f577905f6339ec0. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 32.07it/s]


In [28]:
dataset = dataset.filter(lambda x: x["damsl_act_tag"] in swda_da_tag_indexes)

100%|██████████| 5/5 [00:00<00:00, 23.99ba/s]


In [37]:
set(dataset["train"]["text"])

{'Yeah <laughter>. # /',
 'thank  you for participating. /',
 "I've talked to a lot of people in Texas. /",
 '{F Uh } yeah /',
 'is it?  /',
 '{F Oh, } is it? /',
 "I'm studying speech pathology. /",
 "it's nice talking to you. /",
 '-- Uh-huh,  /',
 'Have fun playing tennis. /',
 "That's one thing I've never really feared. /",
 '<laughter> # right. # /',
 "# <Cough>. #  {D Well, } [ that's, + that ] area over there is not as expensive as in [ the, + the ] immediate, {D you know, } Washington area. /",
 'Okay. /',
 '{D Well, } not being a --',
 'Is that right <laughter>? /',
 'I know it <laughter>. /',
 "You're pretty Texan. /",
 '{D We-, } [ just, + just  ] sentencing, not the trial itself. /',
 'GREEN CARD. /',
 '# Huh.  #',
 "[ I think we've, + I think it's ] come to an  (( eye ))  conclusion  /",
 '# It was nice talking to you. # /',
 "(( That's ))  right. /",
 '<Noise>  Yeah,  /',
 '{D Now, } mine does not work on cars,  /',
 '<TV> {D Well, } ourselves, we have <sigh>, I guess nev

In [44]:
import yaml
import pandas as pd

In [42]:
with open("generics_current_product.yml") as file:
    generics = yaml.safe_load(file)

In [49]:
intents = []
texts = []
for generic in generics:
    for e in generic["examples"]:
        intents.append(generic["intent"])
        texts.append(e["text"])

df = pd.DataFrame({"intent": intents, "text": texts})
df.to_csv("current_product_generics.csv")